### 3.2 创建特性   
特性是一个函数，可以被调用，而不仅仅是用于存储对象的引用    
和属性的另一个区别是，可以很容易给对象添加属性，而不可以轻易的为已有对象添加属性    

特性的操作  
- setter 设置
- getter 获取
- deleter 删除

有两种方法创建特性   
- @property 修饰符
- property() 函数   

关于特性的基本设计模式   
- Eager Calculation(主动计算)： 每当更新特性值时，其他相关特性值都会被立即重新计算  
- Lazy Calculation(延迟计算)： 仅仅当访问特性时，才会触发计算过程 



In [1]:
# 牌定义
class Card:
    def __init__(self, rank, suit, hard, soft):
        """
        rank: 号码 
        suit: 花色 
        hard: 
        soft
        """
        self.rank = rank 
        self.suit = suit 
        self.hard = hard 
        self.soft = soft 

class AceCard(Card):
    def __init__(self, rank, suit):
        super().__init__('A', suit, 1, 11)
    
class NumberCard(Card):
    def __init__(self, rank, suit):
        super().__init__(str(rank), suit, int(rank), int(rank))

class FaceCard(Card):
    def __init__(self, rank, suit):
        super().__init__({11: 'J', 12: 'Q', 13: 'K'}.get(int(rank)), suit, 10, 10)

In [2]:
# 花色定义 
class Suit:
    def __init__(self, name, symbol):
        """
        name: 花色名
        symbol: 花色标识
        """
        self.name = name 
        self.symbol = symbol 

Spade, Heart, Club, Diamond = Suit('Spade', '♠'), Suit('Heart', '♥'), Suit('Club', '♣'), Suit('Diamond', '♦')

In [3]:
# 工厂方法
def card(rank, suit):
    """
    rank: 牌号
    suti: 花色
    """
    if rank == 1: return AceCard(rank, suit) 
    elif 1 < rank <= 10: return NumberCard(rank, suit)
    elif 10 < rank <= 13: return FaceCard(rank, suit) 
    else: raise Exception ('rank is invalid : %s' % rank) 

# 牌堆类
import random 
class Deck(list):
    def __init__(self, decks=1):
        """ decks 有几副牌
        """
        super().__init__()
        for i in range(decks):
            self.extend([card(rank, suit) for rank in range(1, 14) for suit in [Spade, Heart, Club, Diamond]])
        random.shuffle(self)

In [4]:

class Hand:
    def __str__(self):
        return ",".join(map(str, self.card))
    
    def __repr__(self):
        return "{__class__.__name__} ({dealer_card!r} {_cards_str})".format(
            __class__ = self.__class__,
            _cards_str = ', '.join(map(str, self.card)),
            **self.__dict__
        )
    
class LazyHand(Hand):
    def __init__(self, dealer_card, *cards):
        self.dealer_card = dealer_card
        self._cards = list(cards)

    @property
    def total(self):
        delta_soft = max(c.soft - c.hard for c in self._cards)
        hard_total = sum(map(int, [c.hard for c in self._cards]))
        if hard_total + delta_soft <= 21: return hard_total + delta_soft 
        return hard_total 
    
    @property
    def card(self):
        return self._cards 
    
    @card.setter
    def card(self, aCard):
        self._cards.append(aCard)
    
    @card.deleter
    def card(self):
        self._cards.pop(-1)

In [6]:
# HandLazy 中 total 被定义为被动触发，每次触发时都会重新扫描每张牌并完成延迟计算，非常耗时
d = Deck()
h = LazyHand(d.pop(), d.pop(), d.pop())
h.total 

21

In [7]:
## 主动计算  
class EagerHand(Hand):
    def __init__(self, dealer_card, *cards):
        pass 